_Lambda School Data Science — Big Data_

# AWS SageMaker

### Links

#### AWS
- The Open Guide to Amazon Web Services: EC2 Basics _(just this one short section!)_ https://github.com/open-guides/og-aws#ec2-basics
- AWS in Plain English https://www.expeditedssl.com/aws-in-plain-english
- Amazon SageMaker » Create an Amazon SageMaker Notebook Instance https://docs.aws.amazon.com/sagemaker/latest/dg/gs-setup-working-env.html
- Amazon SageMaker » Install External Libraries https://docs.aws.amazon.com/sagemaker/latest/dg/nbi-add-external.html 

`conda install -n python3 bokeh dask datashader fastparquet numba python-snappy`

#### Dask
- Why Dask? https://docs.dask.org/en/latest/why.html
- Use Cases https://docs.dask.org/en/latest/use-cases.html
- User Interfaces https://docs.dask.org/en/latest/user-interfaces.html

#### Numba
- A ~5 minute guide http://numba.pydata.org/numba-doc/latest/user/5minguide.html

## 1. Estimate pi
https://en.wikipedia.org/wiki/Approximations_of_π#Summing_a_circle's_area

### With plain Python

In [1]:
import random

def monte_carlo_pi(nsamples):
    acc = 0
    for _ in range(int(nsamples)):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [2]:
%%time
monte_carlo_pi(1e7)

CPU times: user 4.22 s, sys: 0 ns, total: 4.22 s
Wall time: 4.22 s


3.142204

### With Numba
http://numba.pydata.org/

In [3]:
from numba import jit

In [4]:
@jit(nopython=True) 
def monte_carlo_pi(nsamples):
    acc = 0
    for _ in range(int(nsamples)):
        x = random.random()
        y = random.random()
        if (x**2 + y**2) < 1.0:
            acc += 1
    return 4.0 * acc / nsamples

In [5]:
%%time
monte_carlo_pi(1e7)

CPU times: user 289 ms, sys: 3.33 ms, total: 293 ms
Wall time: 482 ms


3.1412664

## 2. Loop a slow function

### With plain Python

In [6]:
from time import sleep

def slow_square(x):
    sleep(1)
    return x**2

In [7]:
%%time
[slow_square(n) for n in range(16)]

CPU times: user 1.38 ms, sys: 256 µs, total: 1.64 ms
Wall time: 16 s


[0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225]

### With Dask
- https://examples.dask.org/delayed.html
- http://docs.dask.org/en/latest/setup/single-distributed.html

In [8]:
from dask import compute, delayed

In [9]:
%time 
compute(delayed(slow_square)(n) for n in range(16))

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.15 µs


([0, 1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225],)

## 3. Analyze millions of Instacart orders

### Download data
https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2

In [10]:
!wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

--2019-06-10 22:43:28--  https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.186.5
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.186.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 205548478 (196M) [application/x-gzip]
Saving to: ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’

instacart_online_gr 100%[===================>] 196.03M  57.9MB/s    in 3.4s    

2019-06-10 22:43:32 (57.9 MB/s) - ‘instacart_online_grocery_shopping_2017_05_01.tar.gz.1’ saved [205548478/205548478]



In [11]:
!tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

instacart_2017_05_01/
instacart_2017_05_01/._aisles.csv
instacart_2017_05_01/aisles.csv
instacart_2017_05_01/._departments.csv
instacart_2017_05_01/departments.csv
instacart_2017_05_01/._order_products__prior.csv
instacart_2017_05_01/order_products__prior.csv
instacart_2017_05_01/._order_products__train.csv
instacart_2017_05_01/order_products__train.csv
instacart_2017_05_01/._orders.csv
instacart_2017_05_01/orders.csv
instacart_2017_05_01/._products.csv
instacart_2017_05_01/products.csv


In [14]:
%cd instacart_2017_05_01

/home/ec2-user/SageMaker/DS-Unit-3-Sprint-3-Big-Data/module1-aws-sagemaker/instacart_2017_05_01


In [15]:
!ls -lh *.csv

-rw-r--r-- 1 ec2-user ec2-user 2.6K May  2  2017 aisles.csv
-rw-r--r-- 1 ec2-user ec2-user  270 May  2  2017 departments.csv
-rw-r--r-- 1 ec2-user ec2-user 551M May  2  2017 order_products__prior.csv
-rw-r--r-- 1 ec2-user ec2-user  24M May  2  2017 order_products__train.csv
-rw-r--r-- 1 ec2-user ec2-user 104M May  2  2017 orders.csv
-rw-r--r-- 1 ec2-user ec2-user 2.1M May  2  2017 products.csv


### With Pandas

#### Load & merge data

In [16]:
import pandas as pd

In [17]:
%%time
order_products = pd.concat([
    pd.read_csv('order_products__prior.csv'), 
    pd.read_csv('order_products__train.csv')])

order_products.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33819106 entries, 0 to 1384616
Data columns (total 4 columns):
order_id             int64
product_id           int64
add_to_cart_order    int64
reordered            int64
dtypes: int64(4)
memory usage: 1.3 GB
CPU times: user 10.5 s, sys: 2.25 s, total: 12.7 s
Wall time: 16.9 s


In [18]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [19]:
products = pd.read_csv('products.csv')
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49688 entries, 0 to 49687
Data columns (total 4 columns):
product_id       49688 non-null int64
product_name     49688 non-null object
aisle_id         49688 non-null int64
department_id    49688 non-null int64
dtypes: int64(3), object(1)
memory usage: 1.5+ MB


In [20]:
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [21]:
%%time
order_products = pd.merge(order_products, products[['product_id', 'product_name']])

CPU times: user 6.16 s, sys: 1.83 s, total: 8 s
Wall time: 7.99 s


In [22]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered,product_name
0,2,33120,1,1,Organic Egg Whites
1,26,33120,5,0,Organic Egg Whites
2,120,33120,13,0,Organic Egg Whites
3,327,33120,5,1,Organic Egg Whites
4,390,33120,28,1,Organic Egg Whites


#### Most popular products?

In [23]:
%time
order_products['product_name'].value_counts().head(10)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.39 µs


Banana                    491291
Bag of Organic Bananas    394930
Organic Strawberries      275577
Organic Baby Spinach      251705
Organic Hass Avocado      220877
Organic Avocado           184224
Large Lemon               160792
Strawberries              149445
Limes                     146660
Organic Whole Milk        142813
Name: product_name, dtype: int64

#### Organic?

In [24]:
%time
order_products['organic'] = order_products['product_name'] .str.contains('Organic')

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 8.58 µs


In [25]:
%time
order_products['organic'].value_counts()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.34 µs


False    23163118
True     10655988
Name: organic, dtype: int64

### With Dask
https://examples.dask.org/dataframe.html

In [26]:
import dask.dataframe as dd
from dask.distributed import Client

#### Load & merge data
https://examples.dask.org/dataframes/01-data-access.html#Read-CSV-files

In [27]:
%time
order_products = dd.read_csv('order_products*.csv')

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 9.78 µs


In [28]:
%time
order_products = dd.merge(order_products, products[['product_id','product_name']])

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 8.58 µs


http://docs.dask.org/en/latest/dataframe-performance.html#persist-intelligently

In [29]:
order_products.head()

,order_id,product_id,add_to_cart_order,reordered,product_name
0,2,33120,1,1,Organic Egg Whites
1,26,33120,5,0,Organic Egg Whites
2,120,33120,13,0,Organic Egg Whites
3,327,33120,5,1,Organic Egg Whites
4,390,33120,28,1,Organic Egg Whites


#### Most popular products?

In [30]:
%%time
order_products['product_name'].value_counts().compute()

CPU times: user 21.4 s, sys: 3.99 s, total: 25.4 s
Wall time: 8.88 s


Banana                                                       491291
Bag of Organic Bananas                                       394930
Organic Strawberries                                         275577
Organic Baby Spinach                                         251705
Organic Hass Avocado                                         220877
Organic Avocado                                              184224
Large Lemon                                                  160792
Strawberries                                                 149445
Limes                                                        146660
Organic Whole Milk                                           142813
Organic Raspberries                                          142603
Organic Yellow Onion                                         117716
Organic Garlic                                               113936
Organic Zucchini                                             109412
Organic Blueberries                             

#### Organic?

In [31]:
%%time
order_products['organic'] = order_products['product_name'].str.contains('Organic')

CPU times: user 3.95 ms, sys: 0 ns, total: 3.95 ms
Wall time: 3.6 ms


In [32]:
%%time
order_products['organic'].value_counts().compute()

CPU times: user 35.4 s, sys: 3.72 s, total: 39.2 s
Wall time: 21.6 s


False    23163118
True     10655988
Name: organic, dtype: int64

## 4. Fit a machine learning model

### Load data

In [33]:
%cd ../ds-predictive-modeling-challenge-data

/home/ec2-user/SageMaker/DS-Unit-3-Sprint-3-Big-Data/module1-aws-sagemaker/ds-predictive-modeling-challenge-data


In [34]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

train_features = pd.read_csv('train_features.csv')
train_labels = pd.read_csv('train_labels.csv')

X_train_numeric = train_features.select_dtypes(np.number)
y_train = train_labels['status_group']

### With 2 cores (like Google Colab)
https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [35]:
model = RandomForestClassifier(n_estimators=200, oob_score=True, n_jobs=2, random_state=42, verbose=1)
model.fit(X_train_numeric, y_train)
print('Out-of-bag score:', model.oob_score_)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    2.7s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:   11.6s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:   11.8s finished


Out-of-bag score: 0.7206228956228956


### With 16 cores (on AWS m4.4xlarge)

In [36]:
model = RandomForestClassifier(n_estimators=200, oob_score=True, n_jobs=-1, random_state=42, verbose=1)
model.fit(X_train_numeric, y_train)
print('Out-of-bag score:', model.oob_score_)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 200 out of 200 | elapsed:    2.4s finished


Out-of-bag score: 0.7206228956228956


## ASSIGNMENT

Revisit a previous assignment or project that had slow speeds or big data.

Make it better with what you've learned today!

You can use `wget` or Kaggle API to get data. Some possibilities include:

- https://www.kaggle.com/c/ds1-predictive-modeling-challenge
- https://www.kaggle.com/ntnu-testimon/paysim1
- https://github.com/mdeff/fma
- https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2 



Also, you can play with [Datashader](http://datashader.org/) and its [example datasets](https://github.com/pyviz/datashader/blob/master/examples/datasets.yml)!

In [37]:
%cd ../taxicab

/home/ec2-user/SageMaker/DS-Unit-3-Sprint-3-Big-Data/module1-aws-sagemaker/taxicab


In [36]:
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-01.csv
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-02.csv
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-03.csv
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-04.csv
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-05.csv
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-06.csv
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-07.csv
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-08.csv
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-09.csv
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-10.csv
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-11.csv
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-12.csv    


--2019-06-10 22:45:29--  https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-01.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.109.61
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.109.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 772098307 (736M) [text/csv]
Saving to: ‘yellow_tripdata_2018-01.csv’

yellow_tripdata_201 100%[===================>] 736.33M  95.4MB/s    in 7.7s    

2019-06-10 22:45:36 (95.3 MB/s) - ‘yellow_tripdata_2018-01.csv’ saved [772098307/772098307]

--2019-06-10 22:45:37--  https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2018-02.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.115.61
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.115.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 748827487 (714M) [text/csv]
Saving to: ‘yellow_tripdata_2018-02.csv’

yellow_tripdata_201 100%[===================>] 714.14M  93.7MB/s    in 8.3s    

2019-06-10 

In [38]:
df = dd.read_csv('yellow_tripdata_2018*.csv')

In [39]:
df.isnull().sum().compute()

VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
dtype: int64

In [40]:
len(df['VendorID'])

102804250

In [41]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2018-01-01 00:21:05,2018-01-01 00:24:23,1,0.5,1,N,41,24,2,4.5,0.5,0.5,0.00,0.0,0.3,5.80
1,1,2018-01-01 00:44:55,2018-01-01 01:03:05,1,2.7,1,N,239,140,2,14.0,0.5,0.5,0.00,0.0,0.3,15.30
2,1,2018-01-01 00:08:26,2018-01-01 00:14:21,2,0.8,1,N,262,141,1,6.0,0.5,0.5,1.00,0.0,0.3,8.30
3,1,2018-01-01 00:20:22,2018-01-01 00:52:51,1,10.2,1,N,140,257,2,33.5,0.5,0.5,0.00,0.0,0.3,34.80
4,1,2018-01-01 00:09:18,2018-01-01 00:27:06,2,2.5,1,N,246,239,1,12.5,0.5,0.5,2.75,0.0,0.3,16.55


In [42]:
def ride_length(df):
    df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'], infer_datetime_format=True )
    df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'], infer_datetime_format=True )
    
    df['trip_length_in_time'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    
    return df

In [43]:
rl = delayed(ride_length)

In [44]:
rl = rl(df)

In [45]:
df_2 = compute(rl)

In [47]:
df_2

(        VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
 0              1  2018-01-01 00:21:05   2018-01-01 00:24:23                1   
 1              1  2018-01-01 00:44:55   2018-01-01 01:03:05                1   
 2              1  2018-01-01 00:08:26   2018-01-01 00:14:21                2   
 3              1  2018-01-01 00:20:22   2018-01-01 00:52:51                1   
 4              1  2018-01-01 00:09:18   2018-01-01 00:27:06                2   
 5              1  2018-01-01 00:29:29   2018-01-01 00:32:48                3   
 6              1  2018-01-01 00:38:08   2018-01-01 00:48:24                2   
 7              1  2018-01-01 00:49:29   2018-01-01 00:51:53                1   
 8              1  2018-01-01 00:56:38   2018-01-01 01:01:05                1   
 9              1  2018-01-01 00:17:04   2018-01-01 00:22:24                1   
 10             1  2018-01-01 00:41:03   2018-01-01 00:46:49                1   
 11             1  2018-01-0

In [49]:
len(df_2[0])

102804250

In [50]:
from sys import getsizeof

In [51]:
getsizeof(df_2)

56

In [51]:
test = pd.read_csv('yellow_tripdata_2018-01.csv')

In [54]:
test['tpep_pickup_datetime'] = pd.to_datetime(test['tpep_pickup_datetime'], infer_datetime_format=True )
test['tpep_dropoff_datetime'] = pd.to_datetime(test['tpep_dropoff_datetime'], infer_datetime_format=True )

In [56]:
%%time
test['tpep_dropoff_datetime'] - test['tpep_pickup_datetime']

CPU times: user 158 ms, sys: 168 ms, total: 327 ms
Wall time: 325 ms


0         00:03:18
1         00:18:10
2         00:05:55
3         00:32:29
4         00:17:48
5         00:03:19
6         00:10:16
7         00:02:24
8         00:04:27
9         00:05:20
10        00:05:46
11        00:24:39
12        00:04:11
13        00:09:33
14        00:15:46
15        00:17:06
16        00:42:33
17        00:04:13
18        00:36:45
19        00:05:56
20        00:19:43
21        00:12:37
22        00:21:13
23        00:26:50
24        00:12:55
25        00:04:34
26        00:22:01
27        00:02:50
28        00:07:14
29        00:06:57
            ...   
8759844   00:03:12
8759845   00:16:19
8759846   00:12:23
8759847   00:11:34
8759848   00:07:30
8759849   00:13:49
8759850   00:07:07
8759851   00:22:58
8759852   00:28:38
8759853   00:21:52
8759854   00:14:16
8759855   00:04:22
8759856   00:30:31
8759857   00:07:37
8759858   00:03:42
8759859   00:19:34
8759860   00:34:59
8759861   00:01:57
8759862   00:06:11
8759863   00:15:29
8759864   00:08:42
8759865   00